# output-elastic

This component creates a container image for dumping data into elastic search

In [1]:
import logging
import os
import sys
import re

#os.environ['create_image']='True'
os.environ['repository']='docker.io/romeokienzler'
os.environ['version']='0.5'
os.environ['name']='claimed-output-elastic'

In [2]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM debian
    RUN apt update && apt install -y python3-pip
    RUN pip install ipython nbformat elasticsearch==7.9.1
    ADD output-elastic.ipynb .
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t `echo $name`:`echo $version` .
    !docker tag `echo $name`:`echo $version` `echo $repository`/`echo $name`:`echo $version`
    !docker push `echo $repository`/`echo $name`:`echo $version`
    !rm Dockerfile

Sending build context to Docker daemon  30.21kB
Step 1/4 : FROM debian
 ---> 54e726b437fb
Step 2/4 : RUN apt update && apt install -y python3-pip
 ---> Using cache
 ---> 88d22e8c84f5
Step 3/4 : RUN pip install ipython nbformat elasticsearch==7.9.1
 ---> Using cache
 ---> 0973aeff9f48
Step 4/4 : ADD output-elastic.ipynb .
 ---> d448c2070759
Successfully built d448c2070759
Successfully tagged claimed-output-elastic:0.5
The push refers to repository [docker.io/romeokienzler/claimed-output-elastic]

29fceabb: Preparing 
a2c4fdc8: Preparing 
c0e285b8: Preparing 
29fceabb: Pushed lready exists kB0.5: digest: sha256:7af95678a0136712ec5e4f066b94324f3d9a318f4e6a82a030841034445b9476 size: 1162


In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
# connection string
connection_string = os.environ.get('connection_string')

#  document to store
document = os.environ.get('document')

# index
index = os.environ.get('index')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.debug('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
es = Elasticsearch([connection_string], verify_certs=False)

In [ ]:
es.index(index=index, body=document)